# Dota API

The first step is to figure out how we can use the Dota API to get a bunch of historical games.



## Basic API Usage

We'll use this repository as a starting point: https://github.com/bryanveloso/dota2.py

It's a little old and unmaintained but so is the SteamAPI!

In [1]:
import os
import json
import time
import datetime
import requests

We need a Steam API Key. Go get it from: https://steamcommunity.com/dev/apikey

If you like living on the edge you can embed it directly in this notebook and pray that you never commit it to GitHub. Otherwise, export it from your `.bashrc` as `STEAM_API_KEY`.

In [2]:
if 'STEAM_API_KEY' not in os.environ:
  print("No API Key :(")
else:
  print("Found API Key.")
  STEAM_API_KEY = os.environ['STEAM_API_KEY']

Found API Key.


In [3]:
base_url = 'https://api.steampowered.com'

In [4]:
def __request(method, path, **kwargs):
  url = base_url + path
  kwargs.setdefault('params', dict()).update(key=STEAM_API_KEY)
  response = requests.request(method, url, **kwargs)
  return response.json()

In [5]:
def get_steam_id(vanity_name, **params):
    path = '/ISteamUser/ResolveVanityURL/v0001'
    params.update(vanityurl=vanity_name)
    response = __request('get', path, params=params).get('response')
    if response and "success" in response and response['success'] == 1:
        return response
    else:
        print("Error:", response)

In [6]:
response = get_steam_id("Permit")
print("Response:", response)
steam_id = response['steamid']

Response: {'steamid': '76561197990194935', 'success': 1}


Nice. So we can successfully hit the API and get my ID back. 

## Get Information About a Player

In [7]:
def get_player_summaries(steam_ids, **params):
  path = '/ISteamUser/GetPlayerSummaries/v0002'
  params.update(steamids=steam_ids)
  return __request('get', path, params=params)

In [8]:
response = get_player_summaries(steam_id)
print(response)

{'response': {'players': [{'steamid': '76561197990194935', 'communityvisibilitystate': 3, 'profilestate': 1, 'personaname': 'Permit', 'profileurl': 'https://steamcommunity.com/id/Permit/', 'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg', 'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_medium.jpg', 'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_full.jpg', 'avatarhash': 'fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb', 'lastlogoff': 1628362491, 'personastate': 0, 'realname': 'Josh', 'primaryclanid': '103582791429988448', 'timecreated': 1180666476, 'personastateflags': 0, 'loccountrycode': 'CA'}]}}


## Get Information About Games

Let's start by getting the most recent games available.

In [9]:
def get_most_recent_matches():
  path = '/IDOTA2Match_570/GetMatchHistory/V001'
  return __request('get', path)

In [10]:
response = get_most_recent_matches()
response

{'result': {'status': 1,
  'num_results': 100,
  'total_results': 500,
  'results_remaining': 400,
  'matches': [{'match_id': 6124985683,
    'match_seq_num': 5126174231,
    'start_time': 1628368236,
    'lobby_type': 14,
    'radiant_team_id': 0,
    'dire_team_id': 0,
    'players': [{'account_id': 4294967295, 'player_slot': 0, 'hero_id': 21},
     {'player_slot': 1, 'hero_id': 14},
     {'player_slot': 2, 'hero_id': 5},
     {'player_slot': 3, 'hero_id': 29},
     {'player_slot': 4, 'hero_id': 35},
     {'player_slot': 128, 'hero_id': 47},
     {'player_slot': 129, 'hero_id': 44},
     {'player_slot': 130, 'hero_id': 2},
     {'player_slot': 131, 'hero_id': 20},
     {'player_slot': 132, 'hero_id': 26}]},
   {'match_id': 6124985155,
    'match_seq_num': 5126173300,
    'start_time': 1628368200,
    'lobby_type': 4,
    'radiant_team_id': 0,
    'dire_team_id': 0,
    'players': [{'account_id': 846151341, 'player_slot': 0, 'hero_id': 53},
     {'player_slot': 1, 'hero_id': 30},
    

Great! It looks like the games are still in progress though. Some of them don't appear to have finished hero selection so they won't be very useful for us. 

Let's try getting older games.

In [11]:
def get_matches_before(match_id, **params):
  path = '/IDOTA2Match_570/GetMatchHistory/V001'
  params.update(start_at_match_id=match_id)
  return __request('get', path, params=params)

In [12]:
response = get_matches_before(5126114401)
response

{'result': {'status': 1,
  'num_results': 0,
  'total_results': 500,
  'results_remaining': 0,
  'matches': []}}

Hmm this isn't working. Maybe it limits it to the most recent 500 games? That would suck. I will revisit this later.

Later: Yeah apparently this is essentially useless.

## Get Match History By Sequence Number

Here's another API endpoint that looks promising. Let's take a look.

In [13]:
def get_match_history_by_seq_num(seq_num, num_matches, **params):
  path = '/IDOTA2Match_570/GetMatchHistoryBySequenceNum/V001'
  params.update(start_at_match_seq_num=seq_num)
  params.update(matches_requested=num_matches)  
  return __request('get', path, params=params)

In [14]:
response = get_match_history_by_seq_num(5126114401, 1)
response

{'result': {'status': 1,
  'matches': [{'players': [{'account_id': 4294967295,
      'player_slot': 0,
      'hero_id': 62,
      'item_0': 90,
      'item_1': 81,
      'item_2': 36,
      'item_3': 267,
      'item_4': 188,
      'item_5': 214,
      'backpack_0': 0,
      'backpack_1': 0,
      'backpack_2': 0,
      'item_neutral': 573,
      'kills': 5,
      'deaths': 6,
      'assists': 18,
      'leaver_status': 0,
      'last_hits': 42,
      'denies': 1,
      'gold_per_min': 391,
      'xp_per_min': 435,
      'level': 18,
      'hero_damage': 12091,
      'tower_damage': 2203,
      'hero_healing': 0,
      'gold': 595,
      'gold_spent': 13465,
      'scaled_hero_damage': 7605,
      'scaled_tower_damage': 982,
      'scaled_hero_healing': 0,
      'ability_upgrades': [{'ability': 5287, 'time': 164, 'level': 1},
       {'ability': 5286, 'time': 348, 'level': 2},
       {'ability': 5285, 'time': 471, 'level': 3},
       {'ability': 5285, 'time': 696, 'level': 4},
       {'

Great! This looks like what we'll need!